<a href="https://colab.research.google.com/github/luciedeghellinck/Deep-Learning-Project/blob/master/Copy_of_Deep_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [1]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from tqdm import tqdm
import numpy as np

from src.data import Dataset
from src.model import CATEModel
from src.training import train, test

ModuleNotFoundError: No module named 'torchvision'

**Parameters**

In [ ]:
n_hidden_layers = 3
dim_hidden_layers = 100
alpha = 0.356
learning_rate = 4.292 * 10**(-4)
batch_size = 256
dropout_rate = 0.2

**Counter-factual cross-validation**

In [ ]:
# Require: a set of candidate CATE predictors M
MLAlgorithms = []
metaLearners = []

# Require: an observational validation dataset V
dataset = dataset(csv_file)
input_size = dataset.size

# Step 1: Train f(X,T) by minimising Eq.12 using V
model = CATEModel(input_size, n_hidden_layers, dim_hidden_layers, alpha)

In [ ]:
# criterion = nn.MSELoss()
criterion = lossFunction()
optimizer = optim.Adam(model.parameters(), lr = learning_rate) #not sure if model.parameters() works since the model depends on the object Module
train_loader = DataLoader(th.cat(dataset.data[:, 0], dataset.data[:, 2]), batch_size = batch_size) #not sure about the first parameter for the dataloaders (I took the X and Y here)
test_loader = DataLoader(th.cat(dataset.data[:, 0], dataset.data[:, 2]), batch_size = batch_size)

epochs = 100 #They don't give the number of epochs... (this is a random number)

#for the moment, the dropout hasn't yet been integrated
for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train(train_loader, model, optimizer, criterion)
    test_loss, test_acc = test(test_loader, model, criterion)

In [ ]:
# Step 2: Estimate the propensity score (done in step 3 since the propensity is used in the definition of tau)
# Step 3: Caculate the plug-in tau of samples in V
tau = [] # will be an n dimensional vector with the plug-in tau for each feature vector.
features = dataset.data[:,0]
treatments = dataset.data[:,1]
outcomes = dataset.data[:,2]
for datapoint in range(dataset.size): # Iterate over each feature vector from the dataset
  f0 = dataset.get_hypothesis(features[datapoint], 0) # Obtain the hypothesis for the case where there is no treatment for this feature vector
  f1 = dataset.get_hypothesis(features[datapoint], 1) # Obtain the hypothesis for the case where there is a treatment for this feature vector
  regression = propensityRegression(dataset) # Obtain the regression function for the propensity for this feature vector
  plugIn = plugInTau(features[datapoint], treatments[datapoint], outcomes[datapoint], f0, f1, regression)
  tau.append(plugIn)

# Step 4: Estimate the performance of candidate predictors in M based on the performance estimator R and tau.
performance = th.empty((5, 5)) # Tensor containing at position [i,j] the performance estimator R relative to the MLAlgorithm i and the metaLearner j
for algo in MLAlgorithms: 
  for learner in metaLearners: 
    candidate = candidatePredictorTau(features, algo, learner) # n dimensional vector with the candidate tau for each feature vector.
    performance[algo, learner] = performanceEstimator(tau, candidate) # Equation 5 I think

best = th.argmin(performance)
bestAlgo = MLAlgorithms[best // 5]
bestLearner = metaLearners[best % 5]